In [7]:
!python -V

Python 3.12.1


In [8]:
import pandas as pd

In [9]:
import pickle

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from sklearn.feature_extraction import DictVectorizer

from sklearn.metrics import root_mean_squared_error

In [12]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/10/13 10:41:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/13 10:41:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='/workspaces/MLOps-zoomcamp/03-orchestration/mlruns/1', creation_time=1760352061597, experiment_id='1', last_update_time=1760352061597, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [15]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    return df

In [16]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [17]:
len(df_train), len(df_val)

(73908, 61921)

In [19]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import xgboost as xgb

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [36]:
from pathlib import Path
models_folder = Path('models')
models_folder.mkdir(exist_ok=True)

In [38]:
with mlflow.start_run():
    best_params = {
    'learning_rate':0.7645973055915408,
    'max_depth':33,
    'min_child_weight':3.3858551804076,
    'objective':'reg:linear',
    'reg_alpha':0.29589767192164507,
    'reg_lambda': 0.0579918889780704,
    'seed':42
    }

    mlflow.log_params(best_params)
    
    booster = xgb.train(
                params=best_params, 
                dtrain=train,
                num_boost_round=30,
                evals=[(valid,"validation")],
                early_stopping_rounds=50
            )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/usr/local/python/3.12.1/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [10:57:45] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:7.20282
[1]	validation-rmse:6.66342
[2]	validation-rmse:6.58679
[3]	validation-rmse:6.56657
[4]	validation-rmse:6.55688
[5]	validation-rmse:6.54307
[6]	validation-rmse:6.52908
[7]	validation-rmse:6.52438
[8]	validation-rmse:6.51553
[9]	validation-rmse:6.50519
[10]	validation-rmse:6.50157
[11]	validation-rmse:6.49656
[12]	validation-rmse:6.49219
[13]	validation-rmse:6.48876
[14]	validation-rmse:6.48626
[15]	validation-rmse:6.48265
[16]	validation-rmse:6.48169
[17]	validation-rmse:6.47789
[18]	validation-rmse:6.47280
[19]	validation-rmse:6.46550
[20]	validation-rmse:6.46151
[21]	validation-rmse:6.45914
[22]	validation-rmse:6.45671
[23]	validation-rmse:6.45323
[24]	validation-rmse:6.45140
[25]	validation-rmse:6.44857
[26]	validation-rmse:6.44712
[27]	validation-rmse:6.44144
[28]	validation-rmse:6.43953
[29]	validation-rmse:6.43827


2025/10/13 10:57:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/python/3.12.1/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [10:57:52] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/13 10:57:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
